In [23]:
import json
import requests
import praw
import numpy
import time

reddit = praw.Reddit(client_id='tiBit4kBeuoOTA', 
                     client_secret='7-HPDa9q92MS0a-AnUVYb72QDzU', 
                     user_agent='TorkilSamStat', 
                     username='TorkSamStat', 
                     password='bbybbyduerinnafor')

# STATIC VARIABLES
base_url = 'https://api.pushshift.io/reddit/search/comment/'
wanted_count_auth = 2
wanted_count_sub = 2
unwanted_users = ["[deleted]", "AutoModerator"]
unwanted_count = len(unwanted_users)
request_limit = 150
request_count = 0
request_sum = 0
sleep_time = 60
    
# FUNCTIONS
def getKeysFromDict(data):
    users = []
    for obj in data:
        if obj['key'] not in unwanted_users:
            users.append(obj['key'])
    return users

def getMostActiveAuthorsInSubReddit(subreddit):
    global request_count
    global request_sum
    
    url = base_url + '?since=100d&subreddit={}&aggs=author&agg_size={}'.format(subreddit, wanted_count_auth + unwanted_count)
    
    request_count += 1
    request_sum += 1
    print("Performing request number (users):\t ", request_count)
    
    response_raw = requests.get(url)
    print(url, response_raw)
    response = response_raw.json()
    #print(response)
    fetched_authors = getKeysFromDict(response['aggs']['author'])
    print("*******************")
    print()
    
    # Throttle requests to prevent 502 error:
    if request_count >= request_limit:
        print("Sleeping for 60 seconds...")
        time.sleep(sleep_time)
        print("Reset request_count...")
        request_count = 0
    
    return fetched_authors

def getMostActiveSubRedditForAuthor(author):
    global request_count 
    global request_sum
    
    url = base_url + '?since=100d&author={}&aggs=subreddit&agg_size={}'.format(author, wanted_count_sub)
    
    request_count += 1
    request_sum += 1
    
    print("Performing request number (subreddits):\t ", request_count)

    
    response_raw = requests.get(url)
    print(url, response_raw)
    response = response_raw.json()
    #print(response)
    fetched_subreddits = getKeysFromDict(response['aggs']['subreddit'])
    for sub in used_subreddits:
            if (sub not in used_subreddits):
                used_subreddits.append(sub)
    print("*******************")
    print()
    
    # Throttle requests to prevent 502 error:
    if request_count >= request_limit:
        print("Sleeping for 60 seconds...")
        time.sleep(sleep_time)
        print("Reset request_count...")
        request_count = 0
    
    return fetched_subreddits

In [25]:
%%time

# Global variables
used_subreddits = []
used_authors = []
start = time.time()


def get_relations(relational_array, subreddit, wanted_subcount):
       
    
    if wanted_subcount == 0:
        return []
    
    # Add subreddit to used_subreddits 
    used_subreddits.append(subreddit)
    
    # Retrieve users from subreddit and add to request_count
    users = getMostActiveAuthorsInSubReddit(subreddit)

    
    # Retrieve subs from active users in subreddit
    for user in users:
        if user not in used_authors:
            subs = getMostActiveSubRedditForAuthor(user)
            used_authors.append(user)
            
            # Enter new level
            for sub in subs:
                if sub != subreddit:
                    relational_array.append([subreddit, sub])
                    
                if sub not in used_subreddits:
                    relational_array.extend(get_relations([], sub, wanted_subcount - 1))
        
    # Formula for number of requests performed seem to be n**x where n = users gotten for subreddit and x = wanted_subcount (level)
    time_elapsed = time.time() - start
    print("Time elapsed:\t", round(time_elapsed,2), "seconds")
    print("Total requests performed:\t", request_sum)
    
    return relational_array


rels = get_relations([], 'The_Donald', 5)
rels

Performing request number (users):	  1
https://api.pushshift.io/reddit/search/comment/?since=100d&subreddit=The_Donald&aggs=author&agg_size=4 <Response [200]>
*******************

Performing request number (subreddits):	  2
https://api.pushshift.io/reddit/search/comment/?since=100d&author=mangoroti&aggs=subreddit&agg_size=2 <Response [200]>
*******************

Performing request number (users):	  3
https://api.pushshift.io/reddit/search/comment/?since=100d&subreddit=florida&aggs=author&agg_size=4 <Response [200]>
*******************

Performing request number (subreddits):	  4
https://api.pushshift.io/reddit/search/comment/?since=100d&author=ExCap2&aggs=subreddit&agg_size=2 <Response [200]>
*******************

Performing request number (users):	  5
https://api.pushshift.io/reddit/search/comment/?since=100d&subreddit=Unemployment&aggs=author&agg_size=4 <Response [200]>
*******************

Performing request number (subreddits):	  6
https://api.pushshift.io/reddit/search/comment/?sinc

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### 